<a href="https://colab.research.google.com/github/DhruthiSRao/EXL2022/blob/main/FRAUD_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

data = {
    'TransactionID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'AccountID': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'TransactionAmount': [500.00, 1500.00, 200.00, 2500.00, 100.00, 1200.00, 4500.00, 300.00, 700.00, 3200.00],
    'TransactionType': ['Purchase', 'Transfer', 'Purchase', 'Transfer', 'Purchase', 'Transfer', 'Transfer', 'Purchase', 'Purchase', 'Transfer'],
    'TransactionTime': ['2025-01-01 10:00:00', '2025-01-01 11:00:00', '2025-01-02 09:30:00', '2025-01-02 15:45:00', '2025-01-03 08:20:00', '2025-01-03 14:10:00', '2025-01-04 13:00:00', '2025-01-04 11:00:00', '2025-01-05 16:30:00', '2025-01-05 12:00:00'],
    'Location': ['New York', 'Los Angeles', 'Chicago', 'New York', 'San Francisco', 'Boston', 'Los Angeles', 'Miami', 'Chicago', 'New York'],
    'Fraud': [0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
}

df = pd.DataFrame(data)
df.to_csv('transactions.csv', index=False)


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load and preprocess data
try:
    data = pd.read_csv('transactions.csv')
except FileNotFoundError:
    print("The transactions.csv file was not found.")
    exit()

# Ensure data types are correct
data['Fraud'] = data['Fraud'].astype(int)

# Convert 'TransactionTime' to datetime, handling potential errors
try:
    data['TransactionTime'] = pd.to_datetime(data['TransactionTime'], errors='coerce')
except Exception as e:
    print(f"Error converting 'TransactionTime': {e}")
    exit()

# Extract features from 'TransactionTime', handling NaT values
data['Hour'] = data['TransactionTime'].dt.hour.fillna(-1).astype(int)  # Fill NaT with -1
data['DayOfWeek'] = data['TransactionTime'].dt.dayofweek.fillna(-1).astype(int)  # Fill NaT with -1

# Define features and target variable
# Drop original 'TransactionTime' and 'TransactionID' columns
X = data.drop(['Fraud', 'TransactionTime', 'TransactionID'], axis=1)
y = data['Fraud']

# Handle categorical features
X = pd.get_dummies(X, columns=['TransactionType', 'Location'], dummy_na=False)  # Handle missing values

# Handle imbalanced data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

# Save the trained

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 4, n_samples = 4